In [1]:
from __future__ import print_function

import os.path

from google.auth.transport.requests import Request 
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaIoBaseDownload


import io

In [2]:
# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/drive.metadata.readonly','https://www.googleapis.com/auth/drive.readonly']


def main():
    """Shows basic usage of the Drive v3 API.
    Prints the names and ids of the first 10 files the user has access to.
    """
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials_2.json', SCOPES)
            creds = flow.run_local_server(host="localhost", port=8070)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    try:
        service = build('drive', 'v3', credentials=creds)

        # Call the Drive v3 API
        results = service.files().list(
            pageSize=10, fields="nextPageToken, files(id, name)").execute()
        items = results.get('files', [])

        if not items:
            print('No files found.')
            return
        print('Files:')
        for item in items:
            print(u'{0} ({1})'.format(item['name'], item['id']))
    except HttpError as error:
        # TODO(developer) - Handle errors from drive API.
        print(f'An error occurred: {error}')


if __name__ == '__main__':
    main()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=555718989229-lqe62v04ajhr48hus0u1s0kloq3aiscn.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8070%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.metadata.readonly+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.readonly&state=GunGgI3Ij8Uw0r4BvtcCASPZKNI2uL&access_type=offline
Files:
photo_2022-03-07_15-09-1.jpg (1M1GAoE4Z3QPBb22jm6IuhNUYGIQnd2E7)
photo_2022-03-07_15-09.jpg (10GLzSxPivUt93XBXsR_qFlpAQosnerPr)
photo_2022-03-07_15-08.jpg (1plNYt52AI3Mrmetd7zfQlkKPE9du-9Jw)
photo_2022-03-07_15-07.jpg (1RUnnyTq6YvJls_jxrdF8oA1RK9t1Ilhy)
photo_2022-03-07_15-06-1.jpg (147LSLS1ZJUbAws5GMTRCDYAJYhZ2I96x)
photo_2022-03-07_15-06.jpg (16R-eQQYrzOQiltOe0iP5fFzFb0kpY7RO)
photo_2022-03-07_15-05.jpg (1Ek9vb1lCSDmRP6cqehS-0bIOeS6lZgzx)
photo_2022-03-07_15-04.jpg (1ZRLvPclaH0UFlKiR-q5VP1gwgQXmh63r)
photo_2022-03-07_15-03-1.jpg (1yf8yB_gf68xE9Y0AtzsEtqak0

In [ ]:
creds = Credentials.from_authorized_user_file('token.json', SCOPES)


In [3]:
import datetime

In [33]:
datetime.datetime.isoformat(datetime.datetime.utcnow())

'2022-02-18T19:24:24.958341'

In [35]:
base_dir = '../2_horiz/orig/'

In [16]:
import glob
from PIL import Image
import tqdm

In [38]:
files =glob.glob(base_dir+'*.jpg')

In [41]:
def get_date(path):
    img = Image.open(path)
    return img._getexif()[306]

In [42]:
files = list(filter(lambda x: os.path.getsize(x)> 5*10**5, files))


In [48]:
files = list(sorted(files, key=lambda x: get_date(x)))


In [49]:
files[0], files[-1]

('../2_horiz/orig/photo_2022-02-14_12-24.jpg',
 '../2_horiz/orig/photo_2022-02-17_15-34.jpg')

In [94]:
img = Image.open(files[-1])
dt = img._getexif()[306]

In [103]:
date, time = dt.split(' ')

date = date.replace(':','-')

dt = date + ' ' + time

dt

dt=datetime.datetime.fromisoformat(dt)

dt

search_dt = (dt + (datetime.datetime.utcnow() - datetime.datetime.now())).isoformat()

In [112]:
search_dt

'2022-02-17T20:34:25.999996'

In [92]:
service = build('drive', 'v3', credentials=creds)

page_token = None

while True:
    # Call the Drive v3 API
    results = service.files().list(
        pageSize=100, 
        fields="nextPageToken, files(*)",
        q=f"'1vRpM2GnlTqRbpsqnOGJBp18QLKdyk_Gh' in parents  and createdTime > '{search_dt.isoformat()}' " +
            " and mimeType='image/jpeg'",
        orderBy="name",
        pageToken=page_token).execute()
    items = results.get('files', [])
    print('page_token',page_token)
    if not items:
        print('No files found.')
    print('Files:')
    for item in tqdm.notebook.tqdm(items):
        print(u'{2} {0} ({1})'.format(item['name'], item['id'], item['createdTime']))
        file_id = item['id']
        request = service.files().get_media(fileId=file_id)
        fh = io.BytesIO()
        downloader = MediaIoBaseDownload(fh, request)
        done = False
        while done is False:
            status, done = downloader.next_chunk()
        f = open(os.path.join(base_dir, item['name']),'wb')
        f.write(fh.getbuffer())
        f.close()
    page_token = results.get('nextPageToken', None)
    if page_token is None:
        break

page_token None
Files:


/home/shmalex/.conda/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:19: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


2022-02-17T20:35:47.302Z photo_2022-02-17_15-33.jpg (1Oe-A4zkpgk4g5NK6mDewKG2lRTiHp50l)
2022-02-17T20:35:51.124Z photo_2022-02-17_15-34.jpg (18PduatpkkzE6Rxkll7uYbL6nblFyFZP1)
2022-02-17T20:37:30.616Z photo_2022-02-17_15-35-1.jpg (168C-cqPt6iLy9UmzPI2-fwwiGFDuyjh3)
2022-02-17T20:37:34.106Z photo_2022-02-17_15-35.jpg (1gjh_izypQ6N2cfihJL-PGPcC6bAE_lpH)
2022-02-17T20:39:02.846Z photo_2022-02-17_15-36.jpg (1Wbc_iCACddvGathSwhGTqnCzoEvjzwwh)
2022-02-17T20:39:06.646Z photo_2022-02-17_15-37.jpg (1DkGLDtEeyxG3kadVpE9P4vXuOA7rkcEY)
2022-02-17T20:40:28.924Z photo_2022-02-17_15-38-1.jpg (1hdD8I0MbUW8hMg-2iGEBAp-C7ibhAsr_)
2022-02-17T20:40:24.767Z photo_2022-02-17_15-38.jpg (19Lf8gHGh-f3ChX5E3tHBSqqXcE-EZ7Xq)
2022-02-17T20:41:43.569Z photo_2022-02-17_15-39.jpg (1kA2RDoEYfsCrn-5wv9Bdp3vz-D-rb92j)
2022-02-17T20:41:47.334Z photo_2022-02-17_15-40.jpg (1mdtKg6CcaXbBRH8MwnqScbTUx_AIvsGc)
2022-02-17T20:43:18.589Z photo_2022-02-17_15-41-1.jpg (1F0zWf-wip-EP6m5SfRF6lz7la9YxCUnm)
2022-02-17T20:43:24.355Z p

2022-02-17T21:44:49.706Z photo_2022-02-17_16-43.jpg (1cvWJwVZNa-oWiAI1xVjeIMrmsh5HhzHy)
2022-02-17T21:46:19.471Z photo_2022-02-17_16-44-1.jpg (1xLxe5gK-K_BCDjsl_Iqx1a7y5PjEn3WZ)
2022-02-17T21:46:23.951Z photo_2022-02-17_16-44.jpg (19OdmoYvyi7JWsSq9dR7CL06KGETnqQwr)
2022-02-17T21:47:46.480Z photo_2022-02-17_16-45.jpg (1Zwyn0EzrBIHeQ04bSnLP2d_I-v_1m9gZ)
2022-02-17T21:47:50.638Z photo_2022-02-17_16-46.jpg (156StBqz8kTD2uB76C5CNa9HE0WMbrIu0)
2022-02-17T21:49:19.554Z photo_2022-02-17_16-47.jpg (1qFsYNh1YiCK2U583qnDozfaxshMFLlLS)
2022-02-17T21:50:46.767Z photo_2022-02-17_16-48-1.jpg (1vmlSgueKMe3OruEx7ju9MNPJn-G1DzQh)

page_token ~!!~AI9FV7QoPi66XBGTEv_xydN5XKnrqWnqiq22kktbNguVpHj6KorCse555k9Osw5R-pD7u6BWVGPQlm14XMefxKHk-bEZCUz3ZZVcgOnBFtZXycNblLrmlJcmQtaEjLr_V5Xhyvb_0ly_gEG6oQoblxV3xlYpnbh1QmHdxqkUgut3kr2W-70hbAzvtkdnaFFjiqfxB4BSnVVZ2oIg4ui-rs8XmGZ64IjjT74-vxWfw2aQB2BEQm0059xBnLxWipfUVEOl74Rn370VHLcnCMiXlPvGpFwqpxae80fHP0IddYJofd1uY6EPv3WQtQLOyk_8Z54AIdFVXvZVmGNClC-w0NpqnttF1Kmjqbtz6n97wQ3k

2022-02-17T21:49:15.184Z photo_2022-02-17_16-48.jpg (1Q2XwffLpYS95VM5jdFtkwAt8zgFEcGwL)
2022-02-17T21:50:50.543Z photo_2022-02-17_16-49.jpg (1MnKJqenU2DJlbxgUYTsK9gJ_vvTLUUM1)
2022-02-17T21:52:15.423Z photo_2022-02-17_16-50.jpg (1R3GGWZhcdkvYT_LH4b9es4dVEG59-sLA)
2022-02-17T21:53:48.994Z photo_2022-02-17_16-51-1.jpg (11-vKbc2RX5Xjy96lZLMF87SzbNFLg7f1)
2022-02-17T21:52:19.931Z photo_2022-02-17_16-51.jpg (19pT_ZZk76EZMKR2K-EIS9uTIcEEp4-jf)
2022-02-17T21:53:45.418Z photo_2022-02-17_16-52.jpg (1m7v-UoIcKx1zm6Rs7kZqMqBhPN713kOu)
2022-02-17T21:55:14.728Z photo_2022-02-17_16-53.jpg (1YkxoOcsNg0rb4tMLh_7ORGF-CdRzs16p)
2022-02-17T21:56:45.473Z photo_2022-02-17_16-54-1.jpg (1AmfPXtauZR3-2wxBxrvTOiaBGJmKX1QI)
2022-02-17T21:55:18.665Z photo_2022-02-17_16-54.jpg (1VZp6C78VvFRsQfGYrYip5AishPMOH0_Z)
2022-02-17T21:56:50.722Z photo_2022-02-17_16-55.jpg (1LDId3ezlHMndbe-2F5r2vlbPvnu78UUj)
2022-02-17T21:58:14.901Z photo_2022-02-17_16-56.jpg (1xYWI_qXGlN4qdugDq8FSt3tPB5gIkVcE)
2022-02-17T21:59:52.316Z pho

KeyboardInterrupt: 

In [88]:
file_id = item['id']
request = service.files().get_media(fileId=file_id)
fh = io.BytesIO()
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print ("Download %d%%." % int(status.progress() * 100))

Download 100%.


In [89]:
status.progress()

1.0

In [9]:
fh

In [11]:
f = open(item['name'],'wb')

In [14]:
f.close()

In [8]:
image = Image.open('../photo_2022-02-20_09-49-1.jpg')

In [9]:
exif = image.info['exif']


In [12]:

# Your picture process here
image = image.rotate(90)
image.save('../test_rotated.jpg', 'JPEG', exif=exif)

In [122]:
m1 = pyexiv2.ImageMetadata('../photo_2022-02-20_09-49-1.jpg')
m1.read()
# modify tags ...
# m1['Exif.Image.Key'] = pyexiv2.ExifTag('Exif.Image.Key', 'value')
m1.modified = True # not sure what this is good for
m2 = pyexiv2.metadata.ImageMetadata( '../photo_2.jpg' )
m2.read() # yes, we need to read the old stuff before we can overwrite it
m1.copy( m2 )
m2.write()

NameError: name 'pyexiv2' is not defined

In [13]:
from PIL import ImageDraw, ImageFont


In [121]:
title_font = ImageFont.truetype('cour.ttf', 30)
img=Image.open('../photo_2022-02-20_09-49-1.jpg')
title_text = img._getexif()[306]
image_editable = ImageDraw.Draw(img)
image_editable.rectangle((10, 15, 360, 46), fill ="#444444")
image_editable.text((15,15), title_text, (255, 255, 255), font=title_font)
img.save(f'../photo_2.jpg')

In [26]:

files = list(glob.glob('../2_horiz/orig//*.jpg'))

In [27]:
len(files)

9956

In [28]:
def get_date(path):
    img = Image.open(path)
    if img._getexif() is None:
        return ''
    return img._getexif()[306]

In [29]:
created = set(map(get_date, files))

In [30]:
len(created)

9956

In [32]:
d = get_date(files[0])
d

'2022:02:20 12:48:10'